In [8]:
# Basic DS stuff
import numpy as np
import pandas as pd
import json

# Web scraping
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import asyncio

# For labeling records, tracking files, and formatting
from datetime import datetime
import time

# For Rick
import pickle

In [9]:
last_digit = 5

In [10]:
# NOTE: Use this cell to reset the scraper, making it forget all records.

# with open(f'../data/raw/game_mapped_users_{last_digit}.pkl', 'wb+') as file :
#     pickle.dump(set([]), file)

# with open(f'../data/raw/recently_played_{last_digit}.pkl', 'wb+') as file :
#     pickle.dump(set([]), file)

In [11]:
# Load data

with open('../data/raw/all_users', 'rb+') as file :
    all_users = set(pickle.load(file))

with open('../data/raw/game_mapped_users.pkl', 'rb+') as file :
    game_mapped_users = pickle.load(file)

with open(f'../data/raw/game_mapped_users_{last_digit}.pkl', 'rb+') as file :
    game_mapped_users_digit = pickle.load(file)
    
# with open('../data/raw/skipped_game_map_users', 'rb+') as file :
#     skipped_users = pickle.load(file)

In [12]:
# Create vars

holding_set = set()

relevant_users = all_users-game_mapped_users

relevant_users = relevant_users-game_mapped_users_digit

unmapped_users = [user for user in relevant_users if int(user[-1])==last_digit]

key = '83A112A706DF72CB0920A20F28F04252'

interval = 10

base_url = 'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={}&steamid={}&format=json'

-------------
### This is the API version

In [13]:
async def get_games(user, url) :
    return (user, urlopen(url))

In [14]:
to_scrape = len(unmapped_users)

for i in range(0, to_scrape, interval) :
    try :
        iter_max = min(i+interval, to_scrape)
        users = [unmapped_users[i] for i in range(i, iter_max)]
        tasks = [get_games(user, base_url.format(key, user)) for user in users]
        responses = await asyncio.gather(*tasks)
        for response in responses :
            user = response[0]
            jason = json.load(response[1])
            games_list = jason['response']
            if len(games_list) > 0 :
                if games_list['total_count'] > 0 :
                    for game in games_list['games'] :
                        appid = game['appid']
                        playtime_2weeks = game['playtime_2weeks']
                        playtime_forever = game['playtime_forever']
                        holding_set.add((user, appid, playtime_2weeks, playtime_forever))
            game_mapped_users.add(user)
    except Exception as e:
        print(e)
        print(datetime.now())
        print('Trying again in 1 minute...')
        time.sleep(60)

    if i % 200 == 0 :
        with open(f'../data/raw/recently_played_{last_digit}.pkl', 'rb+') as file :
            total_set = pickle.load(file)
        total_set.update(holding_set)
        with open(f'../data/raw/recently_played_{last_digit}.pkl', 'wb+') as file :
            pickle.dump(total_set, file)
        with open(f'../data/raw/game_mapped_users_{last_digit}.pkl', 'wb+') as file :
            pickle.dump(game_mapped_users, file)
        print(f'Added: {len(holding_set)}')
        print(f"Saved so far: {len(total_set)}")
        holding_set = set()
        print(datetime.now())
        print('')


Added: 10
Saved so far: 943
2024-04-03 17:54:18.713751

Added: 231
Saved so far: 1174
2024-04-03 17:54:50.262192

Added: 175
Saved so far: 1349
2024-04-03 17:55:19.400161

Added: 140
Saved so far: 1489
2024-04-03 17:55:57.747395

Added: 207
Saved so far: 1696
2024-04-03 17:56:26.501925

Added: 193
Saved so far: 1889
2024-04-03 17:56:55.969811

Added: 292
Saved so far: 2181
2024-04-03 17:57:24.044904

Added: 267
Saved so far: 2448
2024-04-03 17:57:53.212687

Added: 192
Saved so far: 2640
2024-04-03 17:58:22.622665

Added: 211
Saved so far: 2851
2024-04-03 17:58:52.380689

Added: 176
Saved so far: 3027
2024-04-03 17:59:21.566544

Added: 198
Saved so far: 3225
2024-04-03 17:59:50.075600

Added: 132
Saved so far: 3357
2024-04-03 18:00:20.924448

Added: 250
Saved so far: 3607
2024-04-03 18:00:50.544856

Added: 243
Saved so far: 3850
2024-04-03 18:01:19.218796

Added: 176
Saved so far: 4026
2024-04-03 18:01:47.065082

Added: 868
Saved so far: 4894
2024-04-03 18:02:15.224576

Added: 186
Saved

In [15]:
# This works but is deprecated. Saved as a potential reference.

# counter = 0
# game_counts = set()

# for i in range(100) :

#     user = all_users[i]

#     url = f'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={key}&steamid={user}&format=json'
#     with urlopen(url) as response :
#         jason = json.load(response)
#     games_list = jason['response']
#     if len(games_list) > 0 :
#         if games_list['total_count'] > 0 :
#             print(jason)
#             print('\n')
#             counter +=1
#             game_counts.add(len(games_list))

# print(f'{counter} users had recently played games. Of them:')
# print(f'Avg games played: {sum(game_counts)/len(game_counts)}')
# print(f'Most games played: {max(game_counts)}')